In [165]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

from pyfuseki import FusekiUpdate
from pyfuseki.utils import RdfUtils
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE
from rdflib import Graph, RDFS, URIRef, Namespace, RDF, Literal

import networkx as nx
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

import copy
import string
import random

In [166]:
class Application(tk.Frame):
  # treeview Widgetの情報を格納する変数
  treeview = None

  def __init__(self, master=None):
    # Windowの初期設定を行う。
    super().__init__(master)
    self.master = master
    self.master.title("Fuseki Brower")
    self.master.geometry("1250x650")
    #self.pack()

    self.prefix_grpah_data = {}
    self.ttl_file = 'ttl_file.ttl'
    self.selected_graph = ''
    self.selected_graph_name = ''
    self.gpm_uri = ''
    self.graph_node_list = []
    self.graph_edge_list = []
    self.graph_label_list = {}
    self.graph_position = {}
    # 表示しているグラフの層を記録
    self.graph_level = ''
    self.create_widgets()

  def create_widgets(self):
    #------------------------------------------------------------------
    # LabelFrame
    #------------------------------------------------------------------
    frame_options = {"bd": 2, "relief": "ridge", "foreground": "green"}

    self.frame_ep_info_list = tk.LabelFrame(self.master, text=" Graph Info", width=500, height=120,\
                                            **frame_options)
    self.frame_ep_info_list.place(x=495,y=0)

    self.frame_hierarchy_list = tk.LabelFrame(self.master, text="Node List", width=500, height=240,\
                                            **frame_options)
    self.frame_hierarchy_list.place(x=495,y=140)

    self.frame_node = tk.LabelFrame(self.master, text="Infromation", width=300, height=300,\
                                    **frame_options)
    self.frame_node.place(x=710, y=0)

    self.frame_node_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                        **frame_options)
    self.frame_node_info.grid(row=0, column=0)

    self.frame_supplement_info = tk.LabelFrame(self.frame_node, width=300, height=300,\
                                              **frame_options)
    self.frame_supplement_info.grid(row=1, column=0)

    self.frame_new_gpm_list = tk.LabelFrame(self.master, text="GPM List", width=300, height=200,\
                                            **frame_options)
    self.frame_new_gpm_list.place(x=495, y=410)

    self.frame_document_list = tk.LabelFrame(self.master, text="Document List", width=150, height=140,\
                                            **frame_options)
    self.frame_document_list.place(x=760, y=410)
    
    self.frame_engineer_list = tk.LabelFrame(self.master, text="Engineer List", width=150, height=140,\
                                            **frame_options)
    self.frame_engineer_list.place(x=910, y=410)
    
    self.frame_lld_list = tk.LabelFrame(self.master, text="LLD List", width=150, height=140,\
                                            **frame_options)
    self.frame_lld_list.place(x=1060, y=410)

    self.frame_button = tk.LabelFrame(self.master, text="Button", width=550, height=70,\
                                      **frame_options)
    self.frame_button.place(x=495, y=550)

    self.frame_relation = tk.LabelFrame(self.master, text="Relation List", width=300, height=300,\
                                    **frame_options)
    self.frame_relation.place(x=1040, y=0)

    #------------------------------------------------------------------
    # listbox widget (Graph Info)
    #------------------------------------------------------------------
    self.scrollbar_ep_info_x = tk.Scrollbar(self.frame_ep_info_list, orient=tk.HORIZONTAL)
    self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
    self.scrollbar_ep_info_y = tk.Scrollbar(self.frame_ep_info_list, orient=tk.VERTICAL)
    self.scrollbar_ep_info_y.pack(side="right", fill="y")

    self.listbox_ep_value = tk.StringVar()
    self.graph_list = self.get_graph()
    self.listbox_ep_value.set(self.graph_list)
    self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable=self.listbox_ep_value, \
                            exportselection=False, xscrollcommand=self.scrollbar_ep_info_x.set,\
                            yscrollcommand=self.scrollbar_ep_info_y.set)  
    self.listbox_ep_info.pack(fill="both")
    self.listbox_ep_info.bind("<<ListboxSelect>>", select_graph)

    self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
    self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)

    #------------------------------------------------------------------
    # listbox widget (Document)
    #------------------------------------------------------------------
    self.scrollbar_document_x = tk.Scrollbar(self.frame_document_list, orient=tk.HORIZONTAL)
    self.scrollbar_document_x.pack(side="bottom", fill="x")
    self.scrollbar_document_y = tk.Scrollbar(self.frame_document_list, orient=tk.VERTICAL)
    self.scrollbar_document_y.pack(side="right", fill="y")

    self.listbox_document_value = tk.StringVar()
    self.graph_list = self.get_graph()
    self.listbox_document_value.set(self.graph_list)
    self.listbox_document = tk.Listbox(self.frame_document_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable='', \
                            exportselection=False, xscrollcommand=self.scrollbar_document_x.set,\
                            yscrollcommand=self.scrollbar_document_y.set)  
    self.listbox_document.pack(fill="both")
    # self.listbox_document.bind("<<ListboxSelect>>", select_graph)

    self.scrollbar_document_x.config(command=self.listbox_document.xview)
    self.scrollbar_document_y.config(command=self.listbox_document.yview)
    #------------------------------------------------------------------
    # listbox widget (Engineer)
    #------------------------------------------------------------------
    self.scrollbar_engineer_x = tk.Scrollbar(self.frame_engineer_list, orient=tk.HORIZONTAL)
    self.scrollbar_engineer_x.pack(side="bottom", fill="x")
    self.scrollbar_engineer_y = tk.Scrollbar(self.frame_engineer_list, orient=tk.VERTICAL)
    self.scrollbar_engineer_y.pack(side="right", fill="y")

    self.listbox_engineer_value = tk.StringVar()
    self.graph_list = self.get_graph()
    self.listbox_engineer_value.set(self.graph_list)
    self.listbox_engineer = tk.Listbox(self.frame_engineer_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable='', \
                            exportselection=False, xscrollcommand=self.scrollbar_engineer_x.set,\
                            yscrollcommand=self.scrollbar_engineer_y.set)  
    self.listbox_engineer.pack(fill="both")
    # self.listbox_engineer.bind("<<ListboxSelect>>", select_graph)

    self.scrollbar_engineer_x.config(command=self.listbox_engineer.xview)
    self.scrollbar_engineer_y.config(command=self.listbox_engineer.yview)
    #------------------------------------------------------------------
    # listbox widget (LLD)
    #------------------------------------------------------------------
    self.scrollbar_lld_x = tk.Scrollbar(self.frame_lld_list, orient=tk.HORIZONTAL)
    self.scrollbar_lld_x.pack(side="bottom", fill="x")
    self.scrollbar_lld_y = tk.Scrollbar(self.frame_lld_list, orient=tk.VERTICAL)
    self.scrollbar_lld_y.pack(side="right", fill="y")

    self.listbox_lld_value = tk.StringVar()
    self.graph_list = self.get_graph()
    self.listbox_lld_value.set(self.graph_list)
    self.listbox_lld = tk.Listbox(self.frame_lld_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable='', \
                            exportselection=False, xscrollcommand=self.scrollbar_lld_x.set,\
                            yscrollcommand=self.scrollbar_lld_y.set)  
    self.listbox_lld.pack(fill="both")
    # self.listbox_lld.bind("<<ListboxSelect>>", select_graph)

    self.scrollbar_lld_x.config(command=self.listbox_lld.xview)
    self.scrollbar_lld_y.config(command=self.listbox_lld.yview)
    #------------------------------------------------------------------
    # treeview widget
    #------------------------------------------------------------------
    self.treeview = ttk.Treeview(self.frame_hierarchy_list)
    # 階層列設定
    self.treeview.column("#0", width=180, minwidth=500)
    # 階層列(ツリーカラム)の見出し設定。階層列とする。
    self.treeview.heading("#0")
    self.treeview.bind("<<TreeviewSelect>>", tree_select)
    self.treeview.grid(row=1, column=1, sticky="nsew")

    tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, orient="vertical",\
                      command=self.treeview.yview)
    tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
    self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

    tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, orient="horizontal",\
                      command=self.treeview.xview)
    tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
    self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)
    #------------------------------------------------------------------
    # label (node info title)
    #------------------------------------------------------------------
    self.node_info_id = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="Id:\t                         ", relief="raised")

    self.node_info_id.grid(row=0, column=0)

    self.node_info_actionType = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="ActionType:\t         ", relief="raised")
    self.node_info_actionType.grid(row=1, column=0)

    self.node_info_height = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingHeight:     ", relief="raised")
    self.node_info_height.grid(row=2, column=0)

    self.node_info_width = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingWidth:      ", relief="raised")
    self.node_info_width.grid(row=3, column=0)

    self.node_info_posx = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingX:\t          ", relief="raised")
    self.node_info_posx.grid(row=4, column=0)

    self.node_info_posy = tk.Label(self.frame_node_info, justify="left", \
      font=("MSゴシック", "8", "bold"), width=20, text="GeoBoundingY:              ", relief="raised")
    self.node_info_posy.grid(row=5, column=0)

    self.node_info_layer = tk.Label(self.frame_node_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Layer:        \t          ", relief="raised")
    self.node_info_layer.grid(row=6, column=0, sticky=tk.N)

    self.node_info_value = tk.Label(self.frame_node_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Value:        \t          ", relief="raised")
    self.node_info_value.grid(row=7, column=0, sticky=tk.N)
    #------------------------------------------------------------------
    # label (node info data)
    #------------------------------------------------------------------
    self.node_info_id_text = tk.StringVar()
    self.id_info = tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_id_text,\
                              width=23, relief="ridge")
    self.id_info.grid(row=0, column=1)

    self.node_info_action_text = tk.StringVar()
    self.action_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_action_text,\
                              width=23, relief="ridge")
    self.action_info.grid(row=1, column=1)

    self.node_info_height_text = tk.StringVar()
    self.height_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_height_text,\
                              width=23, relief="ridge")
    self.height_info.grid(row=2, column=1)

    self.node_info_width_text = tk.StringVar()
    self.width_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_width_text,\
                              width=23, relief="ridge")
    self.width_info.grid(row=3, column=1)

    self.node_info_posx_text = tk.StringVar()
    self.posx_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_posx_text,\
                              width=23, relief="ridge")
    self.posx_info.grid(row=4, column=1)

    self.node_info_posy_text = tk.StringVar()
    self.posy_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_posy_text,\
                              width=23, relief="ridge")
    self.posy_info.grid(row=5, column=1)

    self.node_info_layer_text = tk.StringVar()
    self.layer_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_layer_text,\
                              width=23, relief="ridge")
    self.layer_info.grid(row=6, column=1)

    self.node_info_value_text = tk.StringVar()
    self.value_info= tk.Label(self.frame_node_info, justify="left", textvariable=self.node_info_value_text,\
                              width=23, wraplength=130, relief="ridge")
    self.value_info.grid(row=7, column=1)
    #------------------------------------------------------------------
    # label (supplement info title)
    #------------------------------------------------------------------
    self.node_info_intention = tk.Label(self.frame_supplement_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Intention:        \t          ", relief="raised")
    self.node_info_intention.grid(row=0, column=0, sticky=tk.N)

    self.node_info_rationale = tk.Label(self.frame_supplement_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Rationale:        \t          ", relief="raised")
    self.node_info_rationale.grid(row=1, column=0, sticky=tk.N)

    self.node_info_annotation = tk.Label(self.frame_supplement_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Annotation:        \t          ", relief="raised")
    self.node_info_annotation.grid(row=2, column=0, sticky=tk.N)

    self.node_info_tool = tk.Label(self.frame_supplement_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Tool:        \t          ", relief="raised")
    self.node_info_tool.grid(row=3, column=0, sticky=tk.N)

    self.node_info_object = tk.Label(self.frame_supplement_info, justify="right", \
      font=("MSゴシック", "8", "bold"), width=20, text="Object:        \t          ", relief="raised")
    self.node_info_object.grid(row=4, column=0, sticky=tk.N)
    #------------------------------------------------------------------
    # label (supplement info data)
    #------------------------------------------------------------------
    self.intention_text = tk.StringVar()
    self.intention_info = tk.Label(self.frame_supplement_info, justify="left", textvariable=self.intention_text,\
                              width=23, wraplength=130, relief="ridge")
    self.intention_info.grid(row=0, column=1)

    self.rationale_text = tk.StringVar()
    self.rationale_info = tk.Label(self.frame_supplement_info, justify="left", textvariable=self.rationale_text,\
                              width=23, wraplength=130, relief="ridge")
    self.rationale_info.grid(row=1, column=1)

    self.annotation_text = tk.StringVar()
    self.annotation_info = tk.Label(self.frame_supplement_info, justify="left", textvariable=self.annotation_text,\
                              width=23, wraplength=130, relief="ridge")
    self.annotation_info.grid(row=2, column=1)

    self.tool_text = tk.StringVar()
    self.tool_info = tk.Label(self.frame_supplement_info, justify="left", textvariable=self.tool_text,\
                              width=23, wraplength=130, relief="ridge")
    self.tool_info.grid(row=3, column=1)

    self.object_text = tk.StringVar()
    self.object_info = tk.Label(self.frame_supplement_info, justify="left", textvariable=self.object_text,\
                              width=23, wraplength=130, relief="ridge")
    self.object_info.grid(row=4, column=1)
    #------------------------------------------------------------------
    # listbox widget (GPM uri Info)
    #------------------------------------------------------------------
    self.scrollbar_gpm_uri_list_x = tk.Scrollbar(self.frame_new_gpm_list, orient=tk.HORIZONTAL)
    self.scrollbar_gpm_uri_list_x.pack(side="bottom", fill="x")
    self.scrollbar_gpm_uri_list_y = tk.Scrollbar(self.frame_new_gpm_list, orient=tk.VERTICAL)
    self.scrollbar_gpm_uri_list_y.pack(side="right", fill="y")

    self.listbox_gpm_uri_value = tk.StringVar()
    self.listbox_gpm_uri_list = tk.Listbox(self.frame_new_gpm_list, height=6, width=30, selectmode=tk.SINGLE,\
                            listvariable=self.listbox_gpm_uri_value, \
                            exportselection=False, xscrollcommand=self.scrollbar_gpm_uri_list_x.set,\
                            yscrollcommand=self.scrollbar_gpm_uri_list_y.set)  
    self.listbox_gpm_uri_list.pack(fill="both")
    #self.listbox_new_gpm_list.bind("<<ListboxSelect>>", uri_select)

    self.scrollbar_gpm_uri_list_x.config(command=self.listbox_gpm_uri_list.xview)
    self.scrollbar_gpm_uri_list_y.config(command=self.listbox_gpm_uri_list.yview)
    #------------------------------------------------------------------
    # listbox widget (relation type)
    #------------------------------------------------------------------
    self.list_relation_type_data = tk.StringVar()
    data = ['use', 'member']
    self.list_relation_type_data.set(data)
    self.list_relation_type = tk.Listbox(self.frame_relation, height=5, \
                            listvariable=self.list_relation_type_data,\
                            selectmode=tk.SINGLE, exportselection=False)
    self.list_relation_type.pack(fill="both")

    #------------------------------------------------------------------
    # Button widget
    #------------------------------------------------------------------
    self.button_gpm_set = tk.Button(self.frame_button, text=" GPM設定 ", width=7, height=2, command=set_gpm)
    self.button_gpm_set.grid(row=1, column=0, padx=5)

    self.button_relation = tk.Button(self.frame_button, text=" 関連付け ", width=7, height=2,\
                        command=set_relation)
    self.button_relation.grid(row=1, column=1, padx=5)

    self.button_export = tk.Button(self.frame_button, text=" Fuseki Export ", width=10, height=2,\
                        command=fuseki_export)
    self.button_export.grid(row=1, column=2, padx=5)

    self.button_document = tk.Button(self.frame_button, text=" 知識文書 ", width=7, height=2,\
                        command=lambda: self.insert_document_list(self))
    self.button_document.grid(row=1, column=3, padx=5)
    
    self.button_engineer = tk.Button(self.frame_button, text=" 技術者 ", width=7, height=2,\
                        command=lambda: self.insert_engineer_list(self))
    self.button_engineer.grid(row=1, column=4, padx=5)
    
    self.button_lld = tk.Button(self.frame_button, text=" 過去事例 ", width=7, height=2,\
                        command=lambda: self.insert_lld_list(self))
    self.button_lld.grid(row=1, column=5, padx=5)
    #------------------------------------------------------------------
    # canvas
    #------------------------------------------------------------------
    self.graph_canvas = tk.Canvas(self.master)
    self.graph_canvas.place(x=0, y=0, width=480, height=600)
    #canvas スクロールバー
    self.canvas_bar_y = tk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
    self.canvas_bar_x = tk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
    self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
    self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
    self.canvas_bar_y.config(command=self.graph_canvas.yview)
    self.canvas_bar_x.config(command=self.graph_canvas.xview)
    self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, xscrollcommand=self.canvas_bar_x.set)
    # Canvasのスクロール範囲を設定
    self.graph_canvas.config(scrollregion=(0, 0, 1000, 1000))
  #------------------------------------------------------------------
  # 検索結果をリストに表示
  #------------------------------------------------------------------  
  def insert_lld_list(self, event):
    self.listbox_lld.delete(0, tk.END)
    lld_action_list = self.fetch_lld_action(app.selected_graph, self.node_info_value_text.get())
    self.listbox_lld.insert(tk.END, *lld_action_list)

  def insert_document_list(self, event):
    self.listbox_document.delete(0, tk.END)
    document_list = self.fetch_document(app.selected_graph, self.node_info_value_text.get())
    print(document_list)
    self.listbox_document.insert(tk.END, *document_list)

  def insert_engineer_list(self, event):
    self.listbox_engineer.delete(0, tk.END)
    engineer_list = self.fetch_engineer(app.selected_graph, self.node_info_value_text.get())
    self.listbox_engineer.insert(tk.END, *engineer_list)
  #------------------------------------------------------------------
  # 階層Topノードを取得
  #------------------------------------------------------------------
  def get_top_node_list(self):

    #最上層アクションを取得
    query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?id ?value
          WHERE {
                graph """ + app.selected_graph_name + """ {
                  ?s ?p pd3:Action;
                    pd3:id ?id;
                    pd3:value ?value.
                  FILTER NOT EXISTS {?s pd3:attribution ?parent}
              }
          }
    """
    self.node_list = {}
    self.node_list["top"] = []
    self.node_id_list = {}
    self.node_id_list["top"] = []
    #★
    self.treeview_item_list = {}
    self.treeview_item_list["top"] = []
    #★end
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      value = result["value"]["value"]
      id = result["id"]["value"]
      self.node_list["top"].append(value)
      self.node_id_list["top"].append(id)

  #------------------------------------------------------------------
  # 階層ノード情報設定
  #------------------------------------------------------------------
  def get_hierarchy_node_list(self, GPM_uri):

    # 元のtreeviewを削除
    if(app.treeview.get_children):
      for item in app.treeview.get_children():
        app.treeview.delete(item)

    #グラフにあるコンテナ数を取得
    query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT (COUNT(?s) AS ?count)
    WHERE {
          graph """ + app.selected_graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        }
    }
    """
    query_result = get_fuseki_data_json(query)
    for result in query_result:
      container_count = int(result["count"]["value"])

    self.get_top_node_list()

    # Top node listに選択したuri以外のnodeを削除
    top_node_list = copy.copy(self.node_list["top"])
    top_node_id_list = copy.copy(self.node_id_list["top"])

    for index, top_node in enumerate(top_node_list):
      if(not top_node in self.top_node_list_uri[GPM_uri]):
        self.node_list["top"].remove(top_node)
        delete_node_id = top_node_id_list[index]
        self.node_id_list["top"].remove(delete_node_id)

    for node in self.node_list["top"]:
      parent = self.treeview.insert("", tk.END, text=node, open=False)
      #★
      self.treeview_item_list["top"].append(parent)
      #★end
      self.get_member_node_list(node, container_count, parent)

  #------------------------------------------------------------------
  # member情報を取得
  #------------------------------------------------------------------
  def get_member_node_list(self, top_node, count, parent_node):

    if(count == 0):
      return

    #★
    top_node = top_node.replace("<br>","")
    #★end
    query_top_node = '"' + top_node + '"'
    query = """
    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
    SELECT ?s
    WHERE {
          graph """ + app.selected_graph_name + """ {
            ?s ?p pd3:Container;
            pd3:value ?value.
        FILTER(?value = """ + query_top_node + """ )
        }
    }
    """

    query_result = get_fuseki_data_json(query)
    if(len(query_result) > 0):
      self.node_list[top_node] = []
      self.node_id_list[top_node] = []
      #★
      self.treeview_item_list[top_node]= []
      #★end
      count -= 1
      for result in query_result:
        container = result["s"]["value"]

      query_2 = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?id ?value
      WHERE {
            graph """ + app.selected_graph_name + """ {
              ?s ?p pd3:Action;
              pd3:attribution ?parent;
              pd3:id ?id;
              pd3:value ?value.
          FILTER(?parent = <""" + container + """> )
          }
      }
      """
      query_result = get_fuseki_data_json(query_2)
      for result in query_result:
        value = result["value"]["value"]
        id = result["id"]["value"]
        self.node_list[top_node].append(value)
        self.node_id_list[top_node].append(id)
        #return self.get_member_node_list(value, count)

      for node in self.node_list[top_node]:
        child = self.treeview.insert(parent_node, tk.END, text=node, open=False)
        #★
        self.treeview_item_list[top_node].append(child)
        #★end
        self.get_member_node_list(node, count, child)
    else:
      return

  #------------------------------------------------------------------
  # Fusekiからgraphを取得
  #------------------------------------------------------------------
  def get_graph(self):
    query= """PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          select distinct ?graph
          where {
                  graph ?graph {
                    ?s ?p ?o;
                  }
          }"""
    graph_list = []

    results = get_fuseki_data_json(query)
    for result in results:
      graph = result["graph"]["value"]
      graph_list.append(graph)

    #print(graph_list)
    return(graph_list)

  #------------------------------------------------------------------
  # New GPM listを取得
  #------------------------------------------------------------------
  def get_gpm_uri_list(self, node_list):

    self.top_node_list_uri = {}
    for node in self.node_list['top']:
      index = 0
      # nodeのuriを取得
      #★
      node_str = '"' + node + '"'
      query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?s ?id
          WHERE {
                graph """ + app.selected_graph_name + """ {
                  ?s ?p pd3:Action;
                  pd3:value ?value;
                  pd3:id ?id
                  FILTER(?value = """ + node_str + """)
                  FILTER NOT EXISTS {?s pd3:attribution ?parent} 
              }
          }
          """
      query_result = get_fuseki_data_json(query)
      for result in query_result:
        temp_subject = result["s"]["value"]
        id = result["id"]["value"]
        node_subject = temp_subject.replace(id, "")
        if(node_subject not in self.top_node_list_uri):
          self.top_node_list_uri[node_subject]=[]
          self.listbox_gpm_uri_list.insert(index, node_subject)
          index += 1
        self.top_node_list_uri[node_subject].append(node)

    #--------------------------------------------
  # FUSEKI操作
  #--------------------------------------------  
  def fetch_from_fuseki(query):

    sparql = SPARQLWrapper("http://localhost:3030/akiyama", returnFormat='json')
    sparql.setQuery(query)
    results = sparql.query().convert()
    return results

  def fetch_gpm(self):
    query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      PREFIX d3: <http://digital-triplet.net/>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      select distinct ?gen
      where {
        graph ?gen {
          ?s ?p ?o;
          pd3:epType "gpm".
        }
      }
    """
    results = get_fuseki_data_json(query)
    gpm_list = []
    for result in results["results"]["bindings"]:
      gpm_list.append(result['gen']['value'])
    return gpm_list
  
  def fetch_action(self, model):
    query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX d3: <http://digital-triplet.net/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?actionName
        where {
        graph <"""+model+"""> {
            ?s a pd3:Action;
            pd3:value ?actionName
            FILTER (?actionName != "Start" && ?actionName != "end" && ?actionName != "End")
        }
        }
    """
    results = get_fuseki_data_json(query)
    print(results)
    action_list = []
    for result in results:
      action_list.append(result['actionName']['value'])
    return action_list

  def fetch_lld_action(self, model, action_name):
    query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
      PREFIX d3: <http://digital-triplet.net/>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      select ?log ?log_action ?log_action_name (COUNT(?event) as ?count)
        where {
          {GRAPH <"""+model+""">
          {
            ?s pd3:value ?action_name.
            filter(?action_name = '"""+action_name+"""')
          }
          GRAPH ?log
          {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
          }
          GRAPH <http://localhost:3030/akiyama/data/event>
          {
            OPTIONAL{?event d3aki:eventType "reference";
                          d3aki:referTo ?log_action}.}
          }
          UNION
          {
            GRAPH <"""+model+""">
          {
            ?s pd3:value ?action_name.
            filter(?action_name = '""" +action_name+"""')
          }
          GRAPH ?log
          {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
          }
          }
        }
        group by ?log ?log_action ?log_action_name 
        order by DESC(?count)
      """
    results = get_fuseki_data_json(query)
    lld_action_list = []
    for result in results:
      lld_action_list.append(result['log_action_name']['value'])
    return lld_action_list

  def fetch_document(self, model, action_name):
    query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
        PREFIX d3: <http://digital-triplet.net/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?documentTitle ?documentLink (COUNT(?event) as ?count)
        where {
        {
            GRAPH <"""+model+""">
            {
            ?s pd3:value ?action_name.
            filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
                ?document d3aki:reference ?log_action;
                        d3aki:documentURI ?documentURI.
            }
            GRAPH <http://localhost:3030/akiyama/data/document>
            {
            OPTIONAL{?documentURI d3aki:linkTo ?documentLink;
                                    d3aki:title ?documentTitle}.
            }
            GRAPH <http://localhost:3030/akiyama/data/event>
            {
            OPTIONAL{?event d3aki:eventType "reference";
                            d3aki:referTo ?documentURI;
                            d3aki:referedTo ?log_action}.
            }
        }
        UNION
        {
            GRAPH <"""+model+""">
            {
            ?s pd3:value ?action_name.
            filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
                ?document d3aki:reference ?log_action;
                        d3aki:documentURI ?documentURI.
            }
            GRAPH <http://localhost:3030/akiyama/data/document>
            {
            OPTIONAL{?documentURI d3aki:linkTo ?documentLink;
                                    d3aki:title ?documentTitle}.
            }
        }
        }
        group by ?documentTitle ?documentLink
        order by DESC(?count)
      """
    results = get_fuseki_data_json(query)
    document_list = []
    for result in results:
      document_list.append(result['documentTitle']['value'])
    return document_list

  def fetch_engineer(self, model, action_name):
    query="""
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
      PREFIX d3: <http://digital-triplet.net/>
      PREFIX dcterms: <http://purl.org/dc/terms/>
      PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
      select ?log ?log_action ?log_action_name ?engineer ?engineerName ?engineerLink (COUNT(?event) as ?count)
      where {
        {
          GRAPH <"""+model+""">
          {
            ?s pd3:value ?action_name.
            filter(?action_name = '"""+action_name+"""')
          }
          GRAPH ?log
          {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
              ?engineer d3aki:practitioner ?log_action;
                        d3aki:engineerURI ?engineerURI;
                        pd3:value ?engineerName.
          }
          GRAPH <http://localhost:3030/akiyama/data/engineer>
          {
            OPTIONAL{?engineerURI d3aki:linkTo ?engineerLink;
                                  }.
          }
          GRAPH <http://localhost:3030/akiyama/data/event>
          {
            OPTIONAL{?event d3aki:eventType "reference";
                            d3aki:referTo ?engineerURI;
                            d3aki:referedTo ?log_action}.
          }
        }
        UNION
        {
          GRAPH <"""+model+""">
          {
            ?s pd3:value ?action_name.
            filter(?action_name = '"""+action_name+"""')
          }
          GRAPH ?log
          {
            ?log_ep pd3:epType "lld".
            ?log_action rdfs:seeAlso ?s;
            pd3:value ?log_action_name.
              ?engineer d3aki:practitioner ?log_action;
                        d3aki:engineerURI ?engineerURI;
                        pd3:value ?engineerName.
          }
          GRAPH <http://localhost:3030/akiyama/data/engineer>
          {
            OPTIONAL{?engineerURI d3aki:linkTo ?engineerLink;
                                  }.
          }
        }
      }
      group by ?log ?log_action ?log_action_name ?engineer ?engineerName ?engineerLink
      order by DESC(?count)
    """
    results = get_fuseki_data_json(query)
    engineer_list = []
    for result in results:
      engineer_list.append(result['engineerName']['value'])
    return engineer_list

In [167]:
def select_graph(event):
  w = event.widget
  select_index = int(w.curselection()[0])
  app.selected_graph = app.graph_list[select_index]
  app.selected_graph_name = "<" + app.selected_graph + ">"

  # treeviewを更新のため、元のtreeviewを削除
  if(app.treeview.get_children):
    for item in app.treeview.get_children():
      app.treeview.delete(item)

  # gpm listの選択をクリア
  app.listbox_gpm_uri_list.delete(0, tk.END)

  # ノード情報をクリア
  app.node_info_id_text.set("")
  app.node_info_action_text.set("")
  app.node_info_height_text.set("")
  app.node_info_width_text.set("")
  app.node_info_posx_text.set("")
  app.node_info_posy_text.set("")
  app.node_info_layer_text.set("")
  app.node_info_value_text.set("")

  app.get_top_node_list()
  #print(app.node_list)
  #print(app.node_id_list)
  app.get_gpm_uri_list(app.node_list)

  # グラフを表示のため、ttlファイルを取得
  query_ttl_file = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      CONSTRUCT { ?s ?p ?o}
      WHERE {
            graph """ + app.selected_graph_name + """ {
                  ?s ?p ?o;
          }
      }
  """
  #print(query_ttl_file)
  get_ttlfile_data(query_ttl_file, app.ttl_file)

In [168]:
def get_ttlfile_data(query, file_path):
  #★
  sparql = SPARQLWrapper("http://localhost:3030/akiyama/sparql")
  #★end
  sparql.setQuery(query)

  sparql.setReturnFormat(TURTLE)
  results = sparql.query().convert()

  with open(file_path, 'w', encoding = 'utf-8') as f:
    #bytes型をdecodeでstr型に変換する
    f.write(results.decode())

In [169]:
def get_graph_data_by_ttlfile(node_id_list, file_path):

  node_list = []
  edge_list = []
  label_list = {}
  position = {}
  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  with open(file_path, 'r', encoding='utf-8') as f:
    g = Graph()
    g.parse(f)

  for s, p, o in g.triples((None, RDF.type, None)):
    # Action
    if(o == pd3.Action):
      for s1, p1, o1 in g.triples((s, pd3.id, None)):
        id = str(o1)
        if(id in node_id_list):
          node_list.append(id)
          for s2, p2, o2 in g.triples((s, pd3.geoBoundingX, None)):
            #★
            postion_x = float(o2)
            #★end
          for s3, p3, o3 in g.triples((s, pd3.geoBoundingY, None)):
            #★
            postion_y = float(o3)
            #★end
          position[id] = [postion_x, 1000-postion_y]
          for s4, p4, o4 in g.triples((s, pd3.value, None)):
            value = str(o4)
            label_list[id] = value
    # Flow
    elif(o == pd3.Flow):
      for s2, p2, o2 in g.triples((s, pd3.source, None)):
        temp_source = o2
      for s3, p3, o3 in g.triples((temp_source, pd3.id, None)):
        source = str(o3)
      for s2, p2, o2 in g.triples((s, pd3.target, None)):
        temp_target = o2
        for s3, p3, o3 in g.triples((temp_target, pd3.id, None)):
          target = str(o3)
      if(source in node_id_list) and (target in node_id_list):
        edge_list.append((source, target, {"color" : "k"}))

  return node_list, edge_list, label_list, position

In [170]:
def tree_select(event):
  # node名を取得
  record_id = app.treeview.focus()
  current_item = app.treeview.item(record_id)["text"]

  #print(record_id)
  #print(app.graph_level)
  #print(current_item)

  #★
  # node idを取得
  for key in app.treeview_item_list.keys():
    for index, item in enumerate(app.treeview_item_list[key]):
      if(item == record_id):
        node_id = app.node_id_list[key][index]
        new_graph_level = key

  # ノード情報表示ラベルをクリア
  clear_info_label()

  # ノード情報を取得(id, actionType, layer, position など)
  get_node_info(current_item, node_id)

  # 補足情報を取得(intention, annotation, rationale, objectなど)
  get_supplement_info(current_item, node_id)

  image_path = "selected_graph.png"
  # 選択したノードの層と表示しているグラフの層が異なる場合、グラフ更新
  if(new_graph_level != app.graph_level):
    app.graph_level = new_graph_level

    (app.graph_node_list, app.graph_edge_list, app.graph_label_list, app.graph_position) = \
      get_graph_data_by_ttlfile(app.node_id_list[app.graph_level], app.ttl_file)
  
  # 画像を設定
  set_image(image_path, app.graph_node_list, app.graph_edge_list, app.graph_label_list, [node_id],\
            app.graph_position)

  # 画像を更新
  selected_image=Image.open(image_path)
  change_image(selected_image)

In [171]:
def clear_info_label():
  app.node_info_id_text.set("")
  app.node_info_action_text.set("")
  app.node_info_height_text.set("")
  app.node_info_width_text.set("")
  app.node_info_posx_text.set("")
  app.node_info_posy_text.set("")
  app.node_info_layer_text.set("")
  app.node_info_value_text.set("")

  app.intention_text.set("")
  app.rationale_text.set("")
  app.annotation_text.set("")
  app.tool_text.set("")
  app.object_text.set("")

In [172]:
def get_node_info(node, node_id):

  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
      select distinct ?s ?id ?actionType ?height ?width ?pos_x ?pos_y ?layer ?type
      where {
            graph """ + app.selected_graph_name + """ {
            ?s pd3:id ?id;
              pd3:actionType ?actionType;
              pd3:geoBoundingHeight ?height;
              pd3:geoBoundingWidth ?width;
              pd3:geoBoundingX ?pos_x;
              pd3:geoBoundingY ?pos_y;
              pd3:layer ?layer;
              rdf:type ?type.
            }
            FILTER(?id = """ + '"' + node_id + '"' + """)
  }"""

  query_result = get_fuseki_data_json(query)

  """
  print(query)
  print(query_result)
  """

  for result in query_result:
    id = result["id"]["value"] 
    action = result["actionType"]["value"]
    height = result["height"]["value"]
    width = result["width"]["value"]
    pos_x = result["pos_x"]["value"]
    pos_y = result["pos_y"]["value"]
    layer = result["layer"]["value"]

  app.node_info_id_text.set(id)
  app.node_info_action_text.set(action)
  app.node_info_height_text.set(height)
  app.node_info_width_text.set(width)
  app.node_info_posx_text.set(pos_x)
  app.node_info_posy_text.set(pos_y)
  app.node_info_layer_text.set(layer)
  app.node_info_value_text.set(node)

In [173]:
def get_supplement_info(node, node_id):

  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?s
      WHERE {
            graph """ + app.selected_graph_name + """ {
              ?s pd3:id ?id.
          FILTER(?id = """ + '"' + node_id + '"' + """)
          }
      }
  """
  query_result = get_fuseki_data_json(query)
  for result in query_result:
    target = result["s"]["value"]

  target = "<" + target + ">"

  #------------------------------------------------------------------
  # intentionを取得
  #------------------------------------------------------------------
  (intention_id, intention_value) = get_supplement_detail_info("intention", target)

  if(intention_id is not None) and (intention_value is not None):
    app.intention_text.set(intention_value)
  
  #------------------------------------------------------------------
  # rationaleを取得
  #------------------------------------------------------------------
  (rationale_id, rationale_value) = get_supplement_detail_info("rationale", target)

  if(rationale_id is not None) and (rationale_value is not None):
    app.rationale_text.set(rationale_value)

  #------------------------------------------------------------------
  # annotationを取得
  #------------------------------------------------------------------
  (annotation_id, annotation_value) = get_supplement_detail_info("annotation", target)

  if(annotation_id is not None) and (annotation_value is not None):
    app.annotation_text.set(annotation_value)

  #------------------------------------------------------------------
  # toolを取得
  #------------------------------------------------------------------
  (tool_id, tool_value) = get_supplement_detail_info("tool/knowledge", target)

  if(tool_value is not None):
    app.tool_text.set(tool_value)

  #------------------------------------------------------------------
  # objectを取得
  #------------------------------------------------------------------
  (object_id, object_value) = get_supplement_detail_info("", target)

  if(object_id is not None) and (object_value is not None):
    app.object_text.set(object_value)

In [174]:
def get_supplement_detail_info(type, taget_id):

  # オブジェクトではない場合
  if(type != ""):
    type = '"'+ type + '"'

    query= """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s ?id ?value
        WHERE {
              graph """ + app.selected_graph_name + """ {
                ?s pd3:target ?target;
                pd3:arcType ?arcType;
                pd3:id ?id;
                pd3:value ?value.
            FILTER(?arcType = """ + type + """)
            FILTER(?target = """ + taget_id + """)
            }
        }
    """
    results = get_fuseki_data_json(query)
    if(len(results) > 0):
      for result in results:
        subject = result["s"]["value"]
        id = result["id"]["value"]
        value = result["value"]["value"]

      return id, value

  # オブジェクトの場合
  else:
    query_2 = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT ?s 
        WHERE {
              graph """ + app.selected_graph_name + """ {
                ?s pd3:target ?target;
                pd3:arcType ?arcType;
                pd3:id ?id;
                pd3:value ?value.
            FILTER(?arcType = "tool/knowledge")
            FILTER(?target = """ + taget_id + """)
            }
        }
    """
    results_2 = get_fuseki_data_json(query_2)
    if(len(results_2) > 0):
      for result in results_2:
        subject = result["s"]["value"]

      subject = "<" + subject + ">"
      object_query = """
          PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
          SELECT ?id ?value
          WHERE {
                graph """ + app.selected_graph_name + """ {
                  ?s ?p pd3:Object;
                  pd3:output ?output;
                  pd3:id ?id;
                  pd3:value ?value.
              FILTER(?output = """ + subject + """)
              }
          }
      """
      object_result = get_fuseki_data_json(object_query)
      if(len(object_result) > 0):
        for result in object_result:
          object_id = result["id"]["value"]
          object_value = result["value"]["value"]

        return object_id, object_value

  return None, None

In [175]:
def get_fuseki_data_json(query):
  sparql = SPARQLWrapper("http://localhost:3030/akiyama", returnFormat='json')
  sparql.setQuery(query)

  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()

  return results["results"]["bindings"]

In [176]:
def set_image(image_path, nodes, edges, labels, select_node, node_postion):
  graph=nx.DiGraph()
  graph.clear()
  plt.clf()

  #頂点を設定
  graph.add_nodes_from(nodes)
  #辺を設定
  graph.add_edges_from(edges)

  color_map = []
  if not select_node:
    color_map = ['tab:blue']
  else:
    for node in graph:
        if node in select_node:
          color_map.append('orange')
        else:
          color_map.append('tab:blue')

  edge_color = [edge["color"] for edge in graph.edges.values()]

  #graph size
  plt.figure(3,figsize=(8,8)) 

  #plt.margins(0.5, 0)
  #node_size default=300
  nx.draw(graph, with_labels=False, node_color=color_map, font_family='IPAexGothic',\
        node_size=500, pos=node_postion, edge_color=edge_color, width=2.0, alpha=0.5)

  nx.draw_networkx_labels(graph, node_postion, labels, font_size=14, font_family='IPAexGothic',\
                          font_color='black')
  #保存
  plt.savefig(image_path)

In [177]:
#--------------------------------------------
# グラフ画像の入り替え処理
#--------------------------------------------
def change_image(img):
  #canvasの書き換え
  app.graph_canvas.photo = ImageTk.PhotoImage(img)
  app.graph_canvas.itemconfig(app.image_on_canvas, image=app.graph_canvas.photo)

In [178]:
def set_gpm():
  selected_GPM_index = app.listbox_gpm_uri_list.curselection()
  if(len(selected_GPM_index)==0):
    messagebox.showerror('Error', 'No uri data selected!')
  else:
    app.gpm_uri= app.listbox_gpm_uri_list.get(selected_GPM_index)

  app.listbox_gpm_uri_list.select_clear(0, tk.END)
  app.listbox_gpm_uri_list.delete(selected_GPM_index)

  # treeviewを表示
  app.get_hierarchy_node_list(app.gpm_uri)

  # グラフを表示
  show_top_graph()

In [179]:
def show_top_graph():
  # グラフ情報を取得
  (app.graph_node_list, app.graph_edge_list, app.graph_label_list, app.graph_position) = \
    get_graph_data_by_ttlfile(app.node_id_list["top"], app.ttl_file)

  # 最上層グラフを表示
  image_path = "top_graph.png"
  app.graph_level = "top"
  set_image(image_path, app.graph_node_list, app.graph_edge_list, app.graph_label_list, [],\
          app.graph_position)
  top_image=Image.open(image_path)

  app.graph_canvas.photo = ImageTk.PhotoImage(top_image)
  app.image_on_canvas = app.graph_canvas.create_image(0, 0, anchor='nw', image=app.graph_canvas.photo)

In [180]:
def set_relation():

  # tree node名を取得
  record_id = app.treeview.focus()
  if(not record_id):
    messagebox.showerror('Error', 'No node selected!')
  else:
    current_item = app.treeview.item(record_id)["text"]

  # member uriを取得
  selected_member_index = app.listbox_gpm_uri_list.curselection()
  if(len(selected_member_index)==0):
    messagebox.showerror('Error', 'No uri data selected!')
  else:
    member_uri= app.listbox_gpm_uri_list.get(selected_member_index)

  # relation typeを取得
  selected_relation_index = app.list_relation_type.curselection()
  if(len(selected_relation_index)==0):
    messagebox.showerror('Error', 'No relation type selected!')
  else:
    relation_type= app.list_relation_type.get(selected_relation_index)

  #print(current_item)
  #print(member_uri)
  #print(relation_type)

  # 選択したノードのidを取得
  for key in app.node_list.keys():
    node_list = app.node_list[key]
    for index, node in enumerate(node_list):
      if(current_item == node):
        current_node_id = app.node_id_list[key][index]
        #print(current_node_id)

  # 選択したノードのuriを取得
  query = """
      PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
      SELECT ?s ?id
      WHERE {
            graph """ + app.selected_graph_name + """ {
              ?s pd3:id ?id
          FILTER(?id = """ + '"' + current_node_id + '"' + """)
          }
      }
  """
  query_result = get_fuseki_data_json(query)
  for result in query_result:
    subject = result["s"]["value"]
    id = result["id"]["value"]
    node_uri = subject.replace(id, "")

  """
  print(current_item)
  print(current_node_id)
  print(node_uri)
  print(member_uri)
  """

  insert_relation(current_item, current_node_id, node_uri, member_uri, relation_type)

  # treeview 更新
  app.get_hierarchy_node_list(app.gpm_uri)

  # graph 更新
  show_top_graph()

  # メンバー関連付けしたuriをGPM listから削除
  app.listbox_gpm_uri_list.select_clear(0, tk.END)
  app.listbox_gpm_uri_list.delete(selected_member_index)

  # relation listに選択した項目をクリア
  app.list_relation_type.select_clear(0, tk.END)

In [181]:
def insert_relation(current_item, node_id, uri, member_uri, type):

  fuseki_insert_data = []
  member_node_list = []
  pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

  if(type == 'member'):
    #--------------------------------------------------------------
    # memberとmemberのlayerを取得
    #--------------------------------------------------------------
    member_query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        SELECT distinct ?node ?layer
        WHERE {
          graph """ + app.selected_graph_name + """ {
            ?node ?p pd3:Action;
                  pd3:layer ?layer
            filter(regex(str(?node),""" + '"' + member_uri + '"' + """)) 
            FILTER NOT EXISTS {?node pd3:attribution ?parent}
          }
        }
    """
    query_result = get_fuseki_data_json(member_query)
    for result in query_result:
      member_node_list.append(result["node"]["value"])
      layer = result["layer"]["value"]

    #--------------------------------------------------------------
    # containerを追加
    #--------------------------------------------------------------
    # container名
    container_name = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(22))

    # RDF.type
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              RDF.type, pd3.Container])
    # containerType
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.containerType, Literal('specialization')])
    # contraction
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.contraction, URIRef(uri)+URIRef(node_id)])
    # id
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.id, Literal(container_name)])
    # geoBoundingHeight
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.geoBoundingHeight, Literal('200')])
    # geoBoundingWidth
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.geoBoundingWidth, Literal('400')])
    # layer
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.layer, Literal(layer)])
    # member
    for member in member_node_list:
      fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.member, URIRef(member)])
    # output
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.output, URIRef(uri)+URIRef(node_id)])
    # value
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_name),\
                              pd3.value, Literal(current_item)])
    #--------------------------------------------------------------
    # expansionを追加
    #--------------------------------------------------------------
    fuseki_insert_data.append([URIRef(uri)+URIRef(current_item),\
                              pd3.expansion, URIRef(member_uri)+URIRef(container_name)])
    #--------------------------------------------------------------
    # memberのattributionを追加
    #--------------------------------------------------------------
    for member in member_node_list:
      fuseki_insert_data.append([URIRef(member), pd3.attribution, \
                                URIRef(member_uri)+URIRef(container_name)])
    #--------------------------------------------------------------
    # container flowを追加
    #--------------------------------------------------------------
    container_flow_name = ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(22))
    # RDF.type
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              RDF.type, pd3.ContainerFlow])
    # arcType
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.arcType, Literal('hierarchization')])
    # id
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.id, Literal(container_flow_name)])
    # layer
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.layer, Literal(layer)])
    # source
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.source, URIRef(member_uri)+URIRef(container_name)])
    # target
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.target, URIRef(uri)+URIRef(node_id)])
    # value
    fuseki_insert_data.append([URIRef(member_uri)+URIRef(container_flow_name),\
                              pd3.value, Literal("")])

  fuseki_insert(fuseki_insert_data)

In [182]:
def fuseki_insert(insert_data):
  fuseki = FusekiUpdate('http://localhost:3030', 'd3')
  g = Graph()

  RdfUtils.add_list_to_graph(g, insert_data)
  spo_str = '\n'.join(
      [f'{s.n3()} {p.n3()} {o.n3()}.' for (s, p, o) in g]
  )

  fuseki_query = """
                PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
                INSERT DATA {
                  GRAPH """ + app.selected_graph_name + "{" + spo_str + """} }"""
  fuseki.run_sparql(fuseki_query)

  messagebox.showinfo('確認', 'Fusekiにインサートしました！')

In [183]:
def fuseki_export():
  if (not app.selected_graph_name):
    messagebox.showerror('Error', 'No graph selected！')
  else:
    file_path = 'fuseki_browser_export.ttl'
    query = """CONSTRUCT {?subject ?predicate ?object}
              WHERE {
                      graph """ + app.selected_graph_name + """ {
                      ?subject ?predicate ?object
                      }
                    }"""
    get_ttlfile_data(query, file_path)
    messagebox.showinfo('確認', 'ttlファイルをエクスポートしました！')

In [184]:
if __name__ == "__main__":
  # Windowを生成する。
  root = tk.Tk()
  app = Application(master=root)
  # Windowをループさせて、継続的にWindow表示させる。
  app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/site-packages/SPARQLWrapper/Wrapper.py", line 1073, in _query
    response = urlopener(request)
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 531, in open
    response = meth(req, response)
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 640, in http_response
    response = self.parent.error(
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 569, in error
    return self._call_chain(*args)
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/Users/LEON/.pyenv/versions/3.8.5/lib/python3.8/urllib/request.py", line 649, in http_error_default
    raise HTT